In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
URM_test=sps.load_npz("drive/MyDrive/recsys2022-main/dataset/processed_data/URM_test_final.npz")

In [5]:
URM_test.shape

(4439967, 23690)

In [6]:
np.ravel(URM_test.sum(axis=1)).shape

(4439967,)

In [7]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [8]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [9]:
is_sorted(Session_ids)

True

In [10]:
dataset=None
dataset_max=None
for file in ["UCF_W","ICBF_W","Graph_W","ICF_W"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"drive/MyDrive/recsys2022-main/dataset/candidates/traditional_recs/final/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

55
loading UCF_W
16350025
loaded UCF_W
0
loading ICBF_W
14855475
loaded ICBF_W
0
loading Graph_W
8321415
loaded Graph_W
0
loading ICF_W
17771428
loaded ICF_W


In [11]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids


In [12]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [13]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [14]:
import ast
submission_df_LSTM=pd.read_csv("drive/MyDrive/recsys2022-main/dataset/candidates/NN/LSTM/final/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [15]:
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")

In [17]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [20]:
dataset.fillna(0,inplace=True)

In [21]:
gc.collect()

2522

In [22]:
dataset_macro_features = pd.read_csv('drive/MyDrive/recsys2022-main/dataset/processed_data/macro_feats_NN_final.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [23]:
dataset.head()

,Item_ID,Score_UCF_W,Score_ICBF_W,Score_Graph_W,Score_ICF_W,Session_Id,score_LSTM,rank_LSTM,Max_Score_UCF_W,Max_Score_ICBF_W,...,Max_Score_ICF_W,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,11,0.034741,0.115033,0.006554,0.311263,61,0.000000,0.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,19,0.017621,0.000000,0.000000,0.000000,61,0.001333,46.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,23,0.040097,0.072503,0.007389,0.325638,61,0.000000,0.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,25,0.024129,0.081094,0.004554,0.006163,61,0.000000,0.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,69,0.053659,0.085104,0.011284,0.338065,61,0.003676,19.0,206.49619,3.357151,...,0.563566,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [24]:
gc.collect()

0

In [25]:
dataset=dataset.sort_values("Session_Id")

In [26]:
gc.collect()

0

In [27]:
import lightgbm

In [28]:
k=10
result = []

In [29]:
submission_df_LSTM

,Session_Id,Item_ID,score_LSTM,rank_LSTM
0,61,1045,0.415334,1
0,61,4574,0.023962,2
0,61,3581,0.014236,3
0,61,4758,0.011078,4
0,61,143,0.010256,5
...,...,...,...,...
49999,4439966,1463,0.001074,96
49999,4439966,105,0.001074,97
49999,4439966,2039,0.001064,98
49999,4439966,3961,0.001059,99


In [30]:
dataset.columns

Index(['Item_ID', 'Score_UCF_W', 'Score_ICBF_W', 'Score_Graph_W',
       'Score_ICF_W', 'Session_Id', 'score_LSTM', 'rank_LSTM',
       'Max_Score_UCF_W', 'Max_Score_ICBF_W', 'Max_Score_Graph_W',
       'Max_Score_ICF_W', 'length_of_session_seconds',
       'avg_time_spent_per_item_seconds',
       'variance_time_spent_per_item_seconds', 'n_seen_items',
       'n_unique_items', 'session_similarity', 'session_similarity_uniques',
       'session_similarity_feats', 'session_similarity_feats_uniques'],
      dtype='object')

In [32]:
for index in range(k):
    model = lightgbm.Booster(model_file=f'drive/MyDrive/recsys2022-main/rankers/model_{index}.txt')
    result.append(model.predict(dataset.drop(["Session_Id","Item_ID"],axis=1)))

result = np.array(result)
result

array([[-3.63822693, -3.63822693, -3.63822693, ..., -2.46501755,
        -3.02088996, -3.03227098],
       [-3.63822693, -3.63822693, -3.63822693, ..., -2.46501755,
        -3.02088996, -3.03227098],
       [-3.37444737, -3.37444737, -3.37444737, ..., -2.31286052,
        -2.85000878, -2.82217287],
       ...,
       [-3.45565253, -3.45565253, -3.45565253, ..., -2.29413018,
        -2.8026833 , -2.90769864],
       [-3.8735466 , -3.8735466 , -3.8735466 , ..., -2.47393457,
        -3.08243381, -2.88609628],
       [-3.6797104 , -3.6797104 , -3.6797104 , ..., -2.35511787,
        -2.95475084, -2.88915977]])

In [33]:
dataset["score"]=np.mean(result ,axis=0)

In [34]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

115

In [35]:
dataset.head()

,Session_Id,Item_ID,score
16350182,61,4976,-3.217583
16350133,61,3738,-3.217583
16350135,61,3779,-3.217583
16350136,61,3824,-3.217583
16350137,61,3853,-3.217583


In [36]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [37]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [38]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [39]:
dataset.iloc[0]

Session_Id                                                   61
Item_ID       [4976, 3738, 3779, 3824, 3853, 3876, 3878, 393...
score         [-3.2175826855156022, -3.2175826855156022, -3....
indices       [154, 34, 69, 68, 553, 319, 396, 164, 560, 305...
Name: 0, dtype: object

In [40]:
dataset.iloc[0]["Item_ID"][536]

4009

In [41]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [42]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [43]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [44]:
dataset

,Session_Id,predictions,rank
0,61,"[1045, 4574, 3586, 3581, 143, 3414, 3126, 1563...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"[3564, 2346, 69, 3607, 4591, 1683, 4587, 3778,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"[4148, 4146, 2330, 1652, 2704, 3326, 3827, 416...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"[4202, 16, 3338, 2769, 1516, 4990, 2448, 2785,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"[3691, 3555, 2970, 3599, 465, 2899, 3441, 2471...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439646,"[270, 2720, 3424, 3618, 1468, 191, 4749, 1827,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439648,"[4145, 3901, 3223, 3550, 724, 2328, 1643, 2491...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439675,"[4865, 3445, 3879, 4966, 2052, 3116, 819, 4418...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439868,"[519, 3414, 1972, 2285, 4220, 3586, 292, 750, ...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [45]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [46]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [47]:
dataset.columns=["session_id","code","rank"]

In [48]:
dataset.head()

,session_id,code,rank
0,61,"(1045, 4574, 3586, 3581, 143, 3414, 3126, 1563...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96,"(3564, 2346, 69, 3607, 4591, 1683, 4587, 3778,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185,"(4148, 4146, 2330, 1652, 2704, 3326, 3827, 416...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224,"(4202, 16, 3338, 2769, 1516, 4990, 2448, 2785,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285,"(3691, 3555, 2970, 3599, 465, 2899, 3441, 2471...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [49]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [50]:
mapping= pd.read_csv('drive/MyDrive/recsys2022-main/dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [51]:
print(len(submission_df))

5000000


In [52]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [53]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [54]:
submission_df.to_csv(
    f'drive/MyDrive/recsys2022-main/final_sub.csv',
    index=False
    )